# Figure: effect of no venous access

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/rega_data/trauma_categories_Rega Pain Study15.09.2025_v2.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/rega_data/rega_physician_list_09102025.xlsx'
meta_medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/medreg_extraction/joined_final_complete_extractions_20251008_221735.xlsx'

In [ ]:
restrict_to_trauma = True
restrict_to_primary = True

In [ ]:
# Load and preprocess data
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)
meta_medic_df = pd.read_excel(meta_medic_data_path)

# Merge with physician data
medic_df['full_name'] = medic_df['Mitglieder mit Einsatzfunktion'].str.replace(' (Flugarzt/Flugärztin)', '')
medic_df.drop_duplicates(subset=['Mitglieder mit Einsatzfunktion'], inplace=True)
medic_df = medic_df.merge(meta_medic_df, how='left', on='full_name')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

# Remove duplicates
data_df = data_df.drop_duplicates(subset=["SNZ Ereignis Nr. "])

# Filter patients with VAS > 3 at scene
data_df = data_df[data_df["VAS_on_scene"] > 3]

print(f"Total patients after filtering: {len(data_df)}")
print(f"Adult patients: {(data_df['Alter '] >= 16).sum()}")
print(f"Pediatric patients: {(data_df['Alter '] < 16).sum()}")

In [ ]:
if restrict_to_trauma:
    n_non_trauma = data_df[data_df['Einteilung (reduziert)'] != 'Unfall'].shape[0]
    print(f'Excluded {n_non_trauma} non-trauma patients')

    # adult non-trauma patients
    n_adult_non_trauma = data_df[(data_df['Einteilung (reduziert)'] != 'Unfall') & (data_df["Alter "] >= 16)].shape[0]
    print(f'Excluded {n_adult_non_trauma} adult non-trauma patients')
    # pediatric non-trauma patients
    n_pediatric_non_trauma = data_df[(data_df['Einteilung (reduziert)'] != 'Unfall') & (data_df["Alter "] < 16)].shape[0]
    print(f'Excluded {n_pediatric_non_trauma} pediatric non-trauma patients')

    data_df = data_df[data_df['Einteilung (reduziert)'] == 'Unfall']

In [ ]:
if restrict_to_primary:
    n_secondary = data_df[data_df['Einsatzart'] != 'Primär'].shape[0]
    print(f'Excluded {n_secondary} secondary transport patients')

    # adult secondary transport patients
    n_adult_secondary = data_df[(data_df['Einsatzart'] != 'Primär') & (data_df["Alter "] >= 16)].shape[0]
    print(f'Excluded {n_adult_secondary} adult secondary transport patients')
    # pediatric secondary transport patients
    n_pediatric_secondary = data_df[(data_df['Einsatzart'] != 'Primär') & (data_df["Alter "] < 16)].shape[0]
    print(f'Excluded {n_pediatric_secondary} pediatric secondary transport patients')
    data_df = data_df[data_df['Einsatzart'] == 'Primär']

# Adult Patients (≥16 years)

In [ ]:
# Prepare adult-only dataset (≥16 years)
from utils.utils import _extract_venous_access_features


adult_df = data_df[data_df['Alter '] >= 16].copy()

# insufficient pain management (VAS on arrival > 3)
adult_df['insufficient_pain_mgmt'] = (adult_df['VAS_on_arrival'] > 3).astype(int)
venous_access_features = _extract_venous_access_features(adult_df["Zugänge"])
adult_df['venous_access_count'] = venous_access_features['venous_access_count']
adult_df['no_venous_access'] = (adult_df['venous_access_count'] == 0).astype(int)

# Create medication dose variables (matching Table 1 approach)
adult_df['fentanyl_dose'] = 0
adult_df['ketamine_dose'] = 0
adult_df['esketamine_dose'] = 0
adult_df['morphine_dose'] = 0
adult_df['Alle Medikamente'] = adult_df['Alle Medikamente'].str.replace(',', ';')  # replace commas with semicolons for consistency
for i, row in adult_df.iterrows():
    if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
        continue
    for analgetic in row['Alle Medikamente'].split(';'):
        if analgetic.strip() == '':
            continue
        # remove mcg or mg from dose
        if '7IE' in analgetic:
                print(f"Skipping dose with 7IE: {analgetic}")
                continue

        analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
        if 'Fentanyl' in analgetic and '/h' not in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip()
            adult_df.at[i, 'fentanyl_dose'] += float(dose) 
        elif 'Fentanyl' in analgetic and '/h' in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
            dose = float(dose) * adult_df.at[i, 'mission_duration']  
            adult_df.at[i, 'fentanyl_dose'] += float(dose)
        elif 'Ketamin' in analgetic or 'Ketamine' in analgetic:
            dose = analgetic.split('Ketamin')[-1].strip()
            adult_df.at[i, 'ketamine_dose'] += float(dose)
        elif 'Esketamin' in analgetic:
            dose = analgetic.split('Esketamin')[-1].strip()
            adult_df.at[i, 'esketamine_dose'] += float(dose)
        elif 'Morphin' in analgetic or 'Morphine' in analgetic:
            dose = analgetic.split('Morphin')[-1].strip()
            adult_df.at[i, 'morphine_dose'] += float(dose)

# fentanyl given
adult_df['fentanyl_given'] = adult_df['fentanyl_dose'] > 0
adult_df['morphine_given'] = adult_df['morphine_dose'] > 0
adult_df['ketamine_given'] = adult_df['ketamine_dose'] > 0
adult_df['esketamine_given'] = adult_df['esketamine_dose'] > 0

# Create combined medication variables
adult_df['any_opiate_dose'] = adult_df['morphine_dose'] + adult_df['fentanyl_dose']
adult_df['any_opiate_given'] = (adult_df['morphine_dose'] > 0) | (adult_df['fentanyl_dose'] > 0)
adult_df['any_ketamine_given'] = (adult_df['ketamine_dose'] > 0) | (adult_df['esketamine_dose'] > 0)

# combination therapy (opiate + ketamine) vs monotherapy
adult_df['opiate_ketamine_combination_therapy'] = (adult_df['any_opiate_given'] & adult_df['any_ketamine_given']).astype(int)

# Create no_analgesic variable: patients who receive no analgesic
# (no_analgesic corresponds to a dose of any_opiate and any_ketamine of 0)
adult_df['no_analgesic'] = ((adult_df['any_opiate_given'] == 0) & (adult_df['any_ketamine_given'] == 0)).astype(int)


In [ ]:
print('IN OVERALL ADULT POPULATION')
# perform a chi2 test to test the association between no_analgesic and no_venous_access
contingency_table = pd.crosstab(adult_df['no_analgesic'], adult_df['no_venous_access'])
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
print("\nChi2 Test for Association between No Analgesic and No Venous Access:")
print("=" * 60)
print(f"Chi2 Statistic: {chi2:.2f}, p-value: {p:.4f}, Degrees of Freedom: {dof}")
print("Contingency Table:")
print(contingency_table)

In [ ]:
print("IN ADULT PATIENTS WITH INSUFFICIENT PAIN MANAGEMENT")
adult_with_insufficient_pain_management_df = adult_df[adult_df["insufficient_pain_mgmt"] == 1]
# perform a chi2 test to test the association between no_analgesic and no_venous_access
contingency_table = pd.crosstab(adult_with_insufficient_pain_management_df['no_analgesic'], adult_with_insufficient_pain_management_df['no_venous_access'])
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
print("\nChi2 Test for Association between No Analgesic and No Venous Access:")
print("=" * 60)
print(f"Chi2 Statistic: {chi2:.2f}, p-value: {p}, Degrees of Freedom: {dof}")
print("Contingency Table:")
print(contingency_table)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#  --- Data prep ---
df = adult_with_insufficient_pain_management_df[['no_analgesic', 'no_venous_access']].copy()

# counts per cell
grouped = (
    df.groupby(['no_venous_access', 'no_analgesic'])
      .size()
      .reset_index(name='count')
)

# proportions within each no_venous_access
grouped['proportion'] = grouped.groupby('no_venous_access')['count'].transform(lambda x: x / x.sum())

# readable labels
venous_access_label_map = {0: 'With venous access', 1: 'No venous access'}
grouped['no_venous_access'] = grouped['no_venous_access'].map(venous_access_label_map)
analgesic_label_map = {0: 'Received analgesia', 1: 'No analgesia'}
grouped['no_analgesic'] = grouped['no_analgesic'].map(analgesic_label_map)

# consistent ordering
venous_access_order = ['With venous access', 'No venous access']
analgesic_order = ['Received analgesia', 'No analgesia']

# pivot counts (rows = x-axis, cols = hue) in plotting order
count_pivot = (
    grouped.pivot(index='no_venous_access', columns='no_analgesic', values='count')
           .reindex(index=venous_access_order, columns=analgesic_order)
)
prop_pivot = (
    grouped.pivot(index='no_venous_access', columns='no_analgesic', values='proportion')
           .reindex(index=venous_access_order, columns=analgesic_order)
)

# --- Plot proportions ---
plt.figure(figsize=(7,5))
ax = sns.barplot(
    data=grouped,
    x='no_venous_access',
    y='proportion',
    hue='no_analgesic',
    order=venous_access_order,
    hue_order=analgesic_order,
    palette=['#0072CE', '#E31837']
)

# --- Label bars with absolute counts (from the pivot, not the heights) ---
# ax.containers is a list of BarContainer objects, one per hue level, in hue_order
for container, hue_level in zip(ax.containers, analgesic_order):
    counts = count_pivot[hue_level].values
    props = prop_pivot[hue_level].values
    labels = [
        f"{p*100:.1f}% (n={int(c)})" if pd.notna(c) else ""
        for p, c in zip(props, counts)
    ]
    ax.bar_label(container, labels=labels, label_type='edge', fontsize=9)


# --- Style ---
# plt.title('Proportion of Analgesic Use by Venous Access')
plt.xlabel('')
plt.ylabel('Proportion of patients')
plt.ylim(0, 1.10)  # room for labels
plt.legend(frameon=False)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y*100:.0f}%'))
# remove top and right spines
sns.despine()
plt.tight_layout()
plt.show()

In [ ]:
fig = ax.get_figure()
# fig.savefig('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/analysis/adult_trauma/no_venous_access_analgesia_proportions.png', dpi=300)